In [ ]:
!pip install torch torchvision
!pip install opencv-python-headless
!pip install albumentations
!pip install nibabel
!pip install matplotlib
!pip install Pillow

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 12.9 MB/s eta 0:00:00


In [ ]:
import glob
import torch
from torch.utils.data import Dataset
import numpy as np
import SimpleITK as sitk
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Subset


class CustomSegmentationDataset(Dataset):

    def __init__(self, root, transformations=None):
        im_nrrd_paths = sorted(glob.glob(f"{root}/Images/*.nrrd"))
        gt_nrrd_paths = sorted(glob.glob(f"{root}/GoldStandard/*.nrrd"))

        numeri_lista = []

        for percorso in gt_nrrd_paths:
            # Estrai il numero alla fine del percorso del file vicino alla 'L'
            numero = int(percorso.split('/')[-1].split('_')[0][1:])
            numeri_lista.append(numero)


        indice_da_rimuovere = numeri_lista.index(442)

        # Rimuovi l'elemento corrispondente all'indice trovato da gt_paths
        elemento_da_rimuovere = gt_nrrd_paths[indice_da_rimuovere]
        gt_nrrd_paths.remove(elemento_da_rimuovere)

        # images_path.append(im_nrrd_paths)
        # gt_paths.append(gt_nrrd_paths)
        self.ims, self.gts, self.orientation = self.get_slices(im_nrrd_paths, gt_nrrd_paths)
        self.transformations = transformations
        self.n_cls = 2

        assert len(self.ims) == len(self.gts) == len(self.orientation)

    def __len__(self):
        return len(self.ims)

    def __getitem__(self, idx):
        im, gt, orientation = self.ims[idx], self.gts[idx], self.orientation[idx]
        if self.transformations:
            im, gt = self.apply_transformations(im, gt)

        # For visualization purposes
        im = self.preprocess_im(im)
        # For the cases when label equals to 2; to avoid CE Loss error
        gt[gt > 1] = 1

        return im.float(), gt.unsqueeze(0).long(), orientation

    def apply_transformations(self, im, gt): transformed = self.transformations(image = im, mask = gt); return transformed["image"], transformed["mask"]

    def get_slices(self, im_nrrd_paths, gt_nrrd_paths):
        ims, gts, orientation = [], [], []

        for index, (im_nrrd, gt_nrrd) in enumerate(zip(im_nrrd_paths, gt_nrrd_paths)):
            print(f"NRRD file number {index + 1} is being converted...")
            nrrd_im_data, nrrd_gt_data = self.read_nrrd(im_nrrd, gt_nrrd)

            # Slicing in axial, coronal, and sagittal directions
            for axis in range(3):
                for idx in range(nrrd_im_data.shape[axis]):
                    if axis == 0:
                        im_slice = nrrd_im_data[idx, :, :]
                        gt_slice = nrrd_gt_data[idx, :, :]
                        orient = 0
                    elif axis == 1:
                        im_slice = nrrd_im_data[:, idx, :]
                        gt_slice = nrrd_gt_data[:, idx, :]
                        orient = 1
                    elif axis == 2:
                        im_slice = nrrd_im_data[:, :, idx]
                        gt_slice = nrrd_gt_data[:, :, idx]
                        orient = 2

                    if len(np.unique(gt_slice)) == 2:
                        # Resize image and mask to 256x256
                        im_resized = sitk.GetArrayFromImage(sitk.Resample(sitk.GetImageFromArray(im_slice), (256, 256)))
                        gt_resized = sitk.GetArrayFromImage(sitk.Resample(sitk.GetImageFromArray(gt_slice), (256, 256)))
                        ims.append(im_resized)
                        gts.append(gt_resized)
                        orientation.append((index, orient))

        return ims, gts, orientation

    def read_nrrd(self, im_path, gt_path):
        im_data = sitk.ReadImage(im_path)
        gt_data = sitk.ReadImage(gt_path)
        return sitk.GetArrayFromImage(im_data), sitk.GetArrayFromImage(gt_data)

    def preprocess_im(self, im):
        # Push background value of -3024 to -1000
        im[im == -3024] = -1000

        # Min-max scaling
        min_val = torch.min(im)
        max_val = torch.max(im)

        # Ensure the min value is not equal to the max value to avoid division by zero
        if min_val != max_val:
            im = (im - min_val) / (max_val - min_val)
        else:
            im = torch.zeros_like(im)

        return im
# Example usage to plot an image before DataLoader creation
def get_dls(root, transformations, bs, split = [0.6, 0.2, 0.2], ns = 4):

    #assert sum(split) == 1., "Sum of the split must be exactly 1"

    ds = CustomSegmentationDataset(root = root, transformations = transformations)
    n_cls = ds.n_cls

    tr_len = int(len(ds) * split[0])
    val_len = int(len(ds) * split[1])
    test_len = len(ds) - (tr_len + val_len)

    ds_len = len(ds)
    indices = list(range(ds_len))

    # Define the lengths for the splits
    tr_len = int(0.6 * ds_len)  # 60% for training
    val_len = int(0.2 * ds_len)  # 20% for validation
    test_len = ds_len - tr_len - val_len  # Remaining 20% for testing

    # Split indices
    tr_indices = indices[:tr_len]
    val_indices = indices[tr_len:tr_len + val_len]
    test_indices = indices[tr_len + val_len:]

    # Create subsets
    tr_ds = Subset(ds, tr_indices)
    val_ds = Subset(ds, val_indices)
    test_ds = Subset(ds, test_indices)

    print(f"\nThere are {len(tr_ds)} number of images in the train set")
    print(f"There are {len(val_ds)} number of images in the validation set")
    print(f"There are {len(test_ds)} number of images in the test set\n")
    # Get dataloaders
    tr_dl  = DataLoader(dataset = tr_ds, batch_size = bs, shuffle = True, num_workers = ns)
    val_dl = DataLoader(dataset = val_ds, batch_size = bs, shuffle = False, num_workers = ns)
    test_dl = DataLoader(dataset = test_ds, batch_size = 1, shuffle = False, num_workers = ns)

    return ds, tr_dl, val_dl, test_dl, n_cls

root = r"/content/drive/MyDrive/Runnare_fusco/NewPatients/NewPatients"
mean, std, im_h, im_w = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225], 256, 256
trans = A.Compose([
    ToTensorV2(transpose_mask=True),
])
ds, tr_dl, val_dl, test_dl, n_cls = get_dls(root = root, transformations = trans, bs = 16)

NRRD file number 1 is being converted...
NRRD file number 2 is being converted...
NRRD file number 3 is being converted...
NRRD file number 4 is being converted...
NRRD file number 5 is being converted...
NRRD file number 6 is being converted...
NRRD file number 7 is being converted...
NRRD file number 8 is being converted...
NRRD file number 9 is being converted...
NRRD file number 10 is being converted...
NRRD file number 11 is being converted...
NRRD file number 12 is being converted...
NRRD file number 13 is being converted...
NRRD file number 14 is being converted...
NRRD file number 15 is being converted...
NRRD file number 16 is being converted...
NRRD file number 17 is being converted...
NRRD file number 18 is being converted...
NRRD file number 19 is being converted...
NRRD file number 20 is being converted...
NRRD file number 21 is being converted...
NRRD file number 22 is being converted...
NRRD file number 23 is being converted...
NRRD file number 24 is being converted...
N

In [ ]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16429 sha256=74ecbc7c48bdabf3c370b0e20f8fd42d659c9023fdea9013f3033544a589d991
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=7c1663fef342cdc1dec8bb7d93d8cdcfd8e8423e5954a7a6a7f192bb4ac12710
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
def plot(rows, cols, count, im, gt = None, title = "Original Image"):

    plt.subplot(rows, cols, count)
    plt.imshow(im.squeeze(0).float()) if gt else plt.imshow((im * 255).cpu().permute(1, 2, 0).numpy().astype("uint8") * 255)
    plt.axis("off"); plt.title(title)

    return count + 1

def visualize(ds, n_ims):

    plt.figure(figsize = (25, 20))
    rows = n_ims // 4; cols = n_ims // rows
    count = 1
    indices = [random.randint(0, len(ds) - 1) for _ in range(n_ims)]

    for idx, index in enumerate(indices):

        if count == n_ims + 1: break

        im, gt,_ = ds[index]

        # First Plot
        count = plot(rows, cols, count, im = im)

        # Second Plot
        count = plot(rows, cols, count, im = gt.squeeze(0), gt = True, title = "GT Mask")

# visualize(tr_dl.dataset, n_ims = 12)

In [ ]:
!pip install segmentation_models_pytorch

# Resnet 50

In [ ]:
import segmentation_models_pytorch as smp, time
model= smp.Unet(
    encoder_name='resnet50',
    encoder_weights="imagenet",
    encoder_depth=5,
    in_channels=1,
    classes=n_cls,
    activation=None,
    )

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 116MB/s]


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)

class Metrics():

    def __init__(self, pred, gt, loss_fn, eps = 1e-10, n_cls = 2):

        self.pred, self.gt = torch.argmax(pred, dim = 1) > 0, gt # (batch, width, height)
        self.loss_fn, self.eps, self.n_cls, self.pred_, self.device = loss_fn, eps, n_cls, pred, device

    def to_contiguous(self, inp): return inp.contiguous().view(-1)

    def PA(self):

        with torch.no_grad():
            match = torch.eq(self.pred, self.gt).int()

        return float(match.sum()) / float(match.numel())

    def mIoU(self):

        with torch.no_grad():

            pred, gt = self.to_contiguous(self.pred), self.to_contiguous(self.gt)

            iou_per_class = []

            for c in range(self.n_cls):

                match_pred = pred == c
                match_gt   = gt == c

                if match_gt.long().sum().item() == 0: iou_per_class.append(np.nan)

                else:

                    intersect = torch.logical_and(match_pred, match_gt).sum().float().item()
                    union = torch.logical_or(match_pred, match_gt).sum().float().item()

                    iou = (intersect + self.eps) / (union + self.eps)
                    iou_per_class.append(iou)

            return np.nanmean(iou_per_class)

    def loss(self): return self.loss_fn(self.pred_, self.gt.squeeze(1))

def tic_toc(start_time = None): return time.time() if start_time == None else time.time() - start_time

In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:
EPOCHS = 5

In [ ]:
def train(model, tr_dl, val_dl, loss_fn, opt, device, epochs, save_prefix, threshold = 0.00, save_path = "saved_models"):

    tr_loss, tr_pa, tr_iou = [], [], []
    val_loss, val_pa, val_iou = [], [], []
    tr_len, val_len = len(tr_dl), len(val_dl)
    best_loss, decrease, not_improve, early_stop_threshold = np.inf, 1, 0, 5
    os.makedirs(save_path, exist_ok = True)

    model.to(device)
    train_start = tic_toc()
    print("Start training process...")

    for epoch in range(1, epochs + 1):
        tic = tic_toc()
        tr_loss_, tr_iou_, tr_pa_ = 0, 0, 0

        model.train()
        print(f"Epoch {epoch} train process is started...")
        for idx, batch in enumerate(tqdm(tr_dl)):
            ims, gts,_ = batch
            ims, gts = ims.to(device), gts.to(device).long()

            preds = model(ims)

            met = Metrics(preds, gts, loss_fn, n_cls = n_cls)
            loss_ = met.loss().requires_grad_()

            tr_iou_ += met.mIoU()
            tr_pa_ += met.PA()
            tr_loss_ += loss_.item()

            loss_.backward()
            opt.step()
            opt.zero_grad()

        print(f"Epoch {epoch} validation process is started...")
        model.eval()
        val_loss_, val_iou_, val_pa_ = 0, 0, 0

        with torch.no_grad():
            for idx, batch in enumerate(tqdm(val_dl)):

                ims, gts,_ = batch
                ims, gts = ims.to(device), gts.to(device)

                preds = model(ims)

                met = Metrics(preds, gts, loss_fn, n_cls = n_cls)

                val_loss_ += met.loss().item()
                val_iou_ += met.mIoU()
                val_pa_ += met.PA()

        print(f"Epoch {epoch} train process is completed.")

        tr_loss_ /= tr_len
        tr_iou_ /= tr_len
        tr_pa_ /= tr_len

        val_loss_ /= val_len
        val_iou_ /=  val_len
        val_pa_ /=   val_len

        print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
        print(f"\nEpoch {epoch} train process results: \n")
        print(f"Train Time         -> {tic_toc(tic):.3f} secs")
        print(f"Train Loss         -> {tr_loss_:.3f}")
        print(f"Train PA           -> {tr_pa_:.3f}")
        print(f"Train IoU          -> {tr_iou_:.3f}")
        print(f"Validation Loss    -> {val_loss_:.3f}")
        print(f"Validation PA      -> {val_pa_:.3f}")
        print(f"Validation IoU     -> {val_iou_:.3f}\n")

        tr_loss.append(tr_loss_)
        tr_iou.append(tr_iou_)
        tr_pa.append(tr_pa_)

        val_loss.append(val_loss_)
        val_iou.append(val_iou_)
        val_pa.append(val_pa_)

        if best_loss > (val_loss_ + threshold):
            print(f"Loss decreased from {best_loss:.3f} to {val_loss_:.3f}!")
            best_loss = val_loss_
            decrease += 1
            if decrease % 2 == 0:
                print("Saving the model with the best loss value...")
                torch.save(model, f"{save_path}/{save_prefix}_best_model.pt")

        else:

            not_improve += 1
            best_loss = val_loss_
            print(f"Loss did not decrease for {not_improve} epoch(s)!")
            if not_improve == early_stop_threshold:
                print(f"Stopping training process becuase loss value did not decrease for {early_stop_threshold} epochs!")
                break
        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")

    print(f"Train process is completed in {(tic_toc(train_start)) / 60:.3f} minutes.")

    return {"tr_loss": tr_loss, "tr_iou": tr_iou, "tr_pa": tr_pa,
            "val_loss": val_loss, "val_iou": val_iou, "val_pa" : val_pa}

device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetResnet50")

Start training process...
Epoch 1 train process is started...


  0%|          | 0/727 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 727/727 [03:57<00:00,  3.06it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [00:27<00:00,  8.81it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 265.444 secs
Train Loss         -> 0.065
Train PA           -> 0.934
Train IoU          -> 0.893
Validation Loss    -> 0.015
Validation PA      -> 0.987
Validation IoU     -> 0.931

Loss decreased from inf to 0.015!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 264.182 secs
Train Loss         -> 0.006
Train PA           -> 0.938
Train IoU          -> 0.969
Validation Loss    -> 0.009
Validation PA      -> 0.988
Validation IoU     -> 0.955

Loss decreased from 0.009 to 0.009!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [03:57<00:00,  3.06it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [00:27<00:00,  8.93it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 265.123 secs
Train Loss         -> 0.004
Train PA           -> 0.938
Train IoU          -> 0.975
Validation Loss    -> 0.012
Validation PA      -> 0.987
Validation IoU     -> 0.945

Loss did not decrease for 2 epoch(s)!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 22.072 minutes.


In [ ]:
class Plot():

    def __init__(self, res):

        self.res = res

        self.visualize(metric1 = "tr_iou", metric2 = "val_iou", label1 = "Train IoU",
                  label2 = "Validation IoU", title = "Mean Intersection Over Union Learning Curve", ylabel = "mIoU Score")

        self.visualize(metric1 = "tr_pa", metric2 = "val_pa", label1 = "Train PA",
                  label2 = "Validation PA", title = "Pixel Accuracy Learning Curve", ylabel = "PA Score")

        self.visualize(metric1 = "tr_loss", metric2 = "val_loss", label1 = "Train Loss",
                  label2 = "Validation Loss", title = "Loss Learning Curve", ylabel = "Loss Value")

    def plot(self, metric, label): plt.plot(self.res[metric], label = label)

    def decorate(self, ylabel, title): plt.title(title); plt.xlabel("Epochs"); plt.ylabel(ylabel); plt.legend(); plt.show()

    def visualize(self, metric1, metric2, label1, label2, title, ylabel):

        plt.figure(figsize=(10, 5))
        self.plot(metric1, label1); self.plot(metric2, label2)
        self.decorate(ylabel, title)


In [ ]:
def inference(dl, model, device, n_ims = 60):

    cols = n_ims // 3; rows = n_ims // cols

    count = 1
    ims, gts, preds = [], [], []
    for idx, data in enumerate(dl):
        im, gt = data

        # Get predicted mask
        with torch.no_grad(): pred = torch.argmax(model(im.to(device)), dim = 1)
        ims.append(im); gts.append(gt); preds.append(pred)

    plt.figure(figsize = (15, 30))
    for idx, (im, gt, pred) in enumerate(zip(ims, gts, preds)):
        if idx == cols: break
        # First plot
        count = plot(cols, rows, count, im.squeeze(0))

        # Second plot
        count = plot(cols, rows, count, im = gt.squeeze(0), gt = True, title = "Ground Truth")

        # Third plot
        count = plot(cols, rows, count, im = pred, title = "Predicted Mask")

In [ ]:
def print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D):
  print("The MIOU for the segmentation over the test set resulted in:")
  print(np.round(np.array(miou_test).mean(),3), " +\- ",np.round(np.array(miou_test).std(),3) )
  print()
  print("The MIOU 3D for the segmentation over the test set resulted in:")
  print(np.round(np.array(list_mioU3D).mean(),3), " +\- ",np.round(np.array(list_mioU3D).std(),3) )
  print()
  print("The Dicescore for the segmentation over the test set resulted in:")
  print(np.round(np.array(dicescores).mean(),3), " +\- ", np.round(np.array(dicescores).std(),3))
  print()
  print("The Dicescore 3D for the segmentation over the test set resulted in:")
  print(np.round(np.array(list_dice3D).mean(),3), " +\- ", np.round(np.array(list_dice3D).std(),3))
  print()
  print("The volume similarity for the segmentation over the test set resulted in:")
  print(np.round(np.array(volume_similarities).mean(),3), " +\- ", np.round(np.array(volume_similarities).std(),3))
  print()

  print("The volume similarity abs for the segmentation over the test set resulted in:")
  print(np.round(np.array(vol_sim_abs).mean(),3), " +\- ", np.round(np.array(vol_sim_abs).std(),3))
  print()

  print()
  print("The time needed for the segmentation of 8 3D images is:")
  print(np.round(execution_time,2), "seconds.")


In [ ]:
import time
import numpy as np
import warnings
from tqdm import tqdm  # Import tqdm for progress bar

# Suppress the specific warning about DataLoader workers
warnings.filterwarnings("ignore", category=UserWarning, message=".*DataLoader.*max number of worker.*")
def calculate_volume_similarity(gtMask,predMask):
    gtMask = np.array(gtMask)
    predMask = np.array(predMask)
    tp = np.sum((gtMask == 1) & (predMask == 1))
    fp = np.sum((gtMask == 0) & (predMask == 1))
    fn = np.sum((gtMask == 1) & (predMask == 0))
    numerator = fn - fp
    numerator2 = abs(numerator)
    denominator = 2*tp + fp + fn
    if denominator == 0:
      return 0,0
    volume_similarity = 1 - (numerator/denominator)
    volume_similarityabs = 1 - (numerator2/denominator)
    return volume_similarity, volume_similarityabs

def calculateScores(gtMask, predMask):
    # Convert to NumPy arrays if they aren't already
    gtMask = np.array(gtMask)
    predMask = np.array(predMask)

    # Calculate the true positives (TP), false positives (FP), and false negatives (FN)
    tp = np.sum((gtMask == 1) & (predMask == 1))
    fp = np.sum((gtMask == 0) & (predMask == 1))
    fn = np.sum((gtMask == 1) & (predMask == 0))

    # Calculate IoU
    iou = tp / (tp + fp + fn)

    dice = np.sum(predMask[gtMask==1]) * 2.0 / (np.sum(predMask) + np.sum(gtMask))

    return iou, dice



def calculate_results(model,test_data):

  miou_test = []
  dicescores = []
  start_time = time.time()
  min_iou = 0
  current_images = []
  current_gt = []
  index = -1
  volume_similarities = []
  volume_similarity_abs = []
  list_mioU3D = []
  list_dice3D = []
  current_axis = -1

  for idx, data in enumerate(tqdm(test_data)):
      im, gt, orientation = data
      new_index, axis = orientation
      if axis != current_axis:
        if index != -1:  # Ensure it's not the first iteration
            vol_sim , vol_sim_abs = calculate_volume_similarity(current_gt, current_images)
            volume_similarities.append(vol_sim)
            volume_similarity_abs.append(vol_sim_abs)
            miou3D, dice3D = calculateScores(current_gt,current_images)
            list_mioU3D.append(miou3D)
            list_dice3D.append(dice3D)

        index = new_index
        current_axis = axis
        current_images = []
        current_gt = []



      # Get predicted mask
      with torch.no_grad():
          pred = torch.argmax(model(im.to(device)), dim=1)

      pred = np.array(pred.squeeze(0).cpu())  # Remove batch dimension
      gt = np.array(gt.squeeze(0).cpu()[0])

      Iou, dice = calculateScores(gt, pred)

      current_images.append(pred)
      current_gt.append(gt)

      miou_test.append(Iou)
      dicescores.append(dice)

  if current_gt and current_images:
          vol_sim , vol_sim_abs = calculate_volume_similarity(current_gt, current_images)
          volume_similarities.append(vol_sim)
          volume_similarity_abs.append(vol_sim_abs)
          miou3D, dice3D = calculateScores(current_gt,current_images)
          list_mioU3D.append(miou3D)
          list_dice3D.append(dice3D)
  end_time = time.time()  # End the timer

  execution_time = end_time - start_time
  return volume_similarities,volume_similarity_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D


In [ ]:
def calculate_group_averages(scores):
    # Convertiamo la lista in un array NumPy per facilitare la manipolazione
    scores_array = np.array(scores)

    # Calcoliamo la media dei gruppi di tre elementi
    num_groups = len(scores_array) // 3
    group_averages = []

    for i in range(num_groups):
        group = scores_array[i*3:(i+1)*3]  # Prendiamo il sottogruppo di tre elementi
        average = np.mean(group)  # Calcoliamo la media del gruppo
        group_averages.append(average)

    return group_averages

In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)

100%|██████████| 3874/3874 [01:25<00:00, 45.38it/s]


The MIOU for the segmentation over the test set resulted in:
0.897  +\-  0.145

The MIOU 3D for the segmentation over the test set resulted in:
0.905  +\-  0.047

The Dicescore for the segmentation over the test set resulted in:
0.937  +\-  0.121

The Dicescore 3D for the segmentation over the test set resulted in:
0.948  +\-  0.03

The volume similarity for the segmentation over the test set resulted in:
1.019  +\-  0.017

The volume similarity abs for the segmentation over the test set resulted in:
0.978  +\-  0.015


The time needed for the segmentation of 8 3D images is:
85.65 seconds.


# Resnet 34

In [ ]:
import segmentation_models_pytorch as smp, time
model= smp.Unet(
    encoder_name='resnet34',
    encoder_weights="imagenet",
    encoder_depth=5,
    in_channels=1,
    classes=n_cls,
    activation=None,
    )

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)


In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetResnet34")

Start training process...
Epoch 1 train process is started...


100%|██████████| 727/727 [02:31<00:00,  4.80it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [00:18<00:00, 13.15it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 170.105 secs
Train Loss         -> 0.044
Train PA           -> 0.936
Train IoU          -> 0.904
Validation Loss    -> 0.010
Validation PA      -> 0.987
Validation IoU     -> 0.952

Loss decreased from inf to 0.010!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 168.898 secs
Train Loss         -> 0.007
Train PA           -> 0.938
Train IoU          -> 0.963
Validation Loss    -> 0.009
Validation PA      -> 0.987
Validation IoU     -> 0.949

Loss decreased from 0.010 to 0.009!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [02:30<00:00,  4.82it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [00:18<00:00, 13.04it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 169.520 secs
Train Loss         -> 0.004
Train PA           -> 0.938
Train IoU          -> 0.975
Validation Loss    -> 0.009
Validation PA      -> 0.987
Validation IoU     -> 0.952

Loss decreased from 0.009 to 0.009!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 14.123 minutes.


In [ ]:
# model = torch.load("saved_models/liver_best_model.pt")
# inference(test_dl, model = model, device = device)

In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)


100%|██████████| 3874/3874 [01:04<00:00, 59.90it/s]


The MIOU for the segmentation over the test set resulted in:
0.898  +\-  0.136

The MIOU 3D for the segmentation over the test set resulted in:
0.92  +\-  0.034

The Dicescore for the segmentation over the test set resulted in:
0.938  +\-  0.113

The Dicescore 3D for the segmentation over the test set resulted in:
0.957  +\-  0.02

The volume similarity for the segmentation over the test set resulted in:
1.023  +\-  0.008

The volume similarity abs for the segmentation over the test set resulted in:
0.975  +\-  0.008


The time needed for the segmentation of 8 3D images is:
65.02 seconds.


#Resnet50 Unet++

In [ ]:
import segmentation_models_pytorch as smp, time
import segmentation_models_pytorch as smp, time
model = smp.UnetPlusPlus(
    encoder_name='resnet50',
    encoder_depth=5,
    encoder_weights="imagenet",
    in_channels=1,
    classes=n_cls,
    activation=None,
    )



In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)


In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetppResnet50")

Start training process...
Epoch 1 train process is started...


100%|██████████| 727/727 [11:01<00:00,  1.10it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [01:14<00:00,  3.28it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 736.164 secs
Train Loss         -> 0.058
Train PA           -> 0.933
Train IoU          -> 0.893
Validation Loss    -> 0.011
Validation PA      -> 0.987
Validation IoU     -> 0.947

Loss decreased from inf to 0.011!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 736.951 secs
Train Loss         -> 0.005
Train PA           -> 0.938
Train IoU          -> 0.971
Validation Loss    -> 0.008
Validation PA      -> 0.988
Validation IoU     -> 0.951

Loss decreased from 0.010 to 0.008!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [11:02<00:00,  1.10it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [01:14<00:00,  3.26it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 737.007 secs
Train Loss         -> 0.006
Train PA           -> 0.938
Train IoU          -> 0.966
Validation Loss    -> 0.013
Validation PA      -> 0.986
Validation IoU     -> 0.930

Loss did not decrease for 1 epoch(s)!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 61.391 minutes.


In [ ]:
# model = torch.load("saved_models/liver_best_model.pt")
# inference(test_dl, model = model, device = device)

In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)


100%|██████████| 3874/3874 [02:20<00:00, 27.58it/s]


The MIOU for the segmentation over the test set resulted in:
0.904  +\-  0.148

The MIOU 3D for the segmentation over the test set resulted in:
0.923  +\-  0.018

The Dicescore for the segmentation over the test set resulted in:
0.939  +\-  0.131

The Dicescore 3D for the segmentation over the test set resulted in:
0.959  +\-  0.01

The volume similarity for the segmentation over the test set resulted in:
1.008  +\-  0.011

The volume similarity abs for the segmentation over the test set resulted in:
0.985  +\-  0.01


The time needed for the segmentation of 8 3D images is:
140.73 seconds.


#Resnet34 Unet++

In [ ]:
import segmentation_models_pytorch as smp, time
import segmentation_models_pytorch as smp, time
model = smp.UnetPlusPlus(
    encoder_name='resnet34',
    encoder_depth=5,
    encoder_weights="imagenet",
    in_channels=1,
    classes=n_cls,
    activation=None,
    )



In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)


In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetppResnet34")

Start training process...
Epoch 1 train process is started...


100%|██████████| 727/727 [05:10<00:00,  2.34it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [00:36<00:00,  6.60it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 347.649 secs
Train Loss         -> 0.026
Train PA           -> 0.938
Train IoU          -> 0.915
Validation Loss    -> 0.009
Validation PA      -> 0.987
Validation IoU     -> 0.947

Loss decreased from inf to 0.009!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 347.898 secs
Train Loss         -> 0.005
Train PA           -> 0.938
Train IoU          -> 0.973
Validation Loss    -> 0.011
Validation PA      -> 0.987
Validation IoU     -> 0.945

Loss did not decrease for 1 epoch(s)!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [05:10<00:00,  2.34it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [00:36<00:00,  6.63it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 347.517 secs
Train Loss         -> 0.005
Train PA           -> 0.938
Train IoU          -> 0.972
Validation Loss    -> 0.008
Validation PA      -> 0.988
Validation IoU     -> 0.953

Loss decreased from 0.011 to 0.008!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 28.975 minutes.


In [ ]:
# model = torch.load("saved_models/liver_best_model.pt")


In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)

100%|██████████| 3874/3874 [01:21<00:00, 47.49it/s]


The MIOU for the segmentation over the test set resulted in:
0.903  +\-  0.145

The MIOU 3D for the segmentation over the test set resulted in:
0.924  +\-  0.035

The Dicescore for the segmentation over the test set resulted in:
0.94  +\-  0.122

The Dicescore 3D for the segmentation over the test set resulted in:
0.959  +\-  0.021

The volume similarity for the segmentation over the test set resulted in:
1.002  +\-  0.016

The volume similarity abs for the segmentation over the test set resulted in:
0.976  +\-  0.017


The time needed for the segmentation of 8 3D images is:
81.86 seconds.


# Unet mobile

In [ ]:
import segmentation_models_pytorch as smp, time
model= smp.Unet(
    encoder_name='timm-mobilenetv3_large_100',
    encoder_depth=5,
    encoder_weights="imagenet",
    in_channels=1,
    classes=n_cls,
    activation=None,
    )


# Assuming you want to load the best model saved during training
  # Make sure this matches the save_path used during training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the best model
# best_model_path = f"/content/drive/MyDrive/saved_models/UnetMobile_best_model.pt"
# model = torch.load(best_model_path, map_location=torch.device(device))


# Make sure to put the model in evaluation mode
#model.eval()



Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_mobilenetv3_large_100-427764d5.pth" to /root/.cache/torch/hub/checkpoints/tf_mobilenetv3_large_100-427764d5.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 241MB/s]


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)


In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetMobile")

Start training process...
Epoch 1 train process is started...


100%|█████████▉| 726/727 [01:59<00:00,  6.18it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 727/727 [02:00<00:00,  6.03it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [00:15<00:00, 15.37it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 136.324 secs
Train Loss         -> 0.049
Train PA           -> 0.933
Train IoU          -> 0.903
Validation Loss    -> 0.011
Validation PA      -> 0.987
Validation IoU     -> 0.945

Loss decreased from inf to 0.011!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 135.918 secs
Train Loss         -> 0.004
Train PA           -> 0.938
Train IoU          -> 0.975
Validation Loss    -> 0.009
Validation PA      -> 0.987
Validation IoU     -> 0.950

Loss decreased from 0.013 to 0.009!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [02:00<00:00,  6.04it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [00:15<00:00, 15.40it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 136.138 secs
Train Loss         -> 0.004
Train PA           -> 0.938
Train IoU          -> 0.975
Validation Loss    -> 0.010
Validation PA      -> 0.987
Validation IoU     -> 0.952

Loss did not decrease for 2 epoch(s)!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 11.352 minutes.


In [ ]:
# model = torch.load("saved_models/liver_best_model.pt")
# inference(test_dl, model = model, device = device)

In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)

100%|██████████| 3874/3874 [01:24<00:00, 45.75it/s]


The MIOU for the segmentation over the test set resulted in:
0.901  +\-  0.128

The MIOU 3D for the segmentation over the test set resulted in:
0.924  +\-  0.025

The Dicescore for the segmentation over the test set resulted in:
0.941  +\-  0.107

The Dicescore 3D for the segmentation over the test set resulted in:
0.96  +\-  0.014

The volume similarity for the segmentation over the test set resulted in:
1.02  +\-  0.011

The volume similarity abs for the segmentation over the test set resulted in:
0.974  +\-  0.012


The time needed for the segmentation of 8 3D images is:
85.04 seconds.


# Unet++ mobile

In [ ]:
import segmentation_models_pytorch as smp, time
model= smp.UnetPlusPlus(
    encoder_name='timm-mobilenetv3_large_100',
    encoder_depth=5,
    encoder_weights="imagenet",
    in_channels=1,
    classes=n_cls,
    activation=None,
    )

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 3e-4)


In [ ]:
import json, os, torch, cv2, random, numpy as np, albumentations as A, nibabel as nib
from matplotlib import pyplot as plt; from glob import glob;
from torch.utils.data import random_split, Dataset, DataLoader
from albumentations.pytorch import ToTensorV2; from PIL import Image
from torchvision import transforms as tfs
from tqdm import tqdm

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

history = train(model = model, tr_dl = tr_dl, val_dl = val_dl,
                 loss_fn = loss_fn, opt = optimizer, device = device,
                 epochs = EPOCHS, save_prefix = "UnetppMobile")

Start training process...
Epoch 1 train process is started...


100%|██████████| 727/727 [02:50<00:00,  4.27it/s]


Epoch 1 validation process is started...


100%|██████████| 243/243 [00:21<00:00, 11.52it/s]


Epoch 1 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 1 train process results: 

Train Time         -> 191.466 secs
Train Loss         -> 0.032
Train PA           -> 0.938
Train IoU          -> 0.911
Validation Loss    -> 0.016
Validation PA      -> 0.987
Validation IoU     -> 0.931

Loss decreased from inf to 0.016!
Saving the model with the best loss value...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 2 train process is started...


...

Epoch 4 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 4 train process results: 

Train Time         -> 190.428 secs
Train Loss         -> 0.004
Train PA           -> 0.938
Train IoU          -> 0.976
Validation Loss    -> 0.011
Validation PA      -> 0.987
Validation IoU     -> 0.949

Loss did not decrease for 1 epoch(s)!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process is started...


100%|██████████| 727/727 [02:49<00:00,  4.28it/s]


Epoch 5 validation process is started...


100%|██████████| 243/243 [00:20<00:00, 11.63it/s]

Epoch 5 train process is completed.

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Epoch 5 train process results: 

Train Time         -> 190.609 secs
Train Loss         -> 0.005
Train PA           -> 0.938
Train IoU          -> 0.974
Validation Loss    -> 0.008
Validation PA      -> 0.988
Validation IoU     -> 0.956

Loss decreased from 0.011 to 0.008!
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Train process is completed in 15.903 minutes.


In [ ]:
# model = torch.load("saved_models/liver_best_model.pt")
# inference(test_dl, model = model, device = device)

In [ ]:
volume_similarities,vol_sim_abs, miou_test, dicescores, execution_time, list_mioU3D,list_dice3D = calculate_results(model,test_dl )

volume_similarities = calculate_group_averages(volume_similarities)
vol_sim_abs = calculate_group_averages(vol_sim_abs)
list_mioU3D = calculate_group_averages(list_mioU3D)
list_dice3D = calculate_group_averages(list_dice3D)
print_results(miou_test,dicescores,volume_similarities,vol_sim_abs,execution_time,list_mioU3D,list_dice3D)

 90%|████████▉ | 3478/3874 [01:22<00:10, 36.82it/s]